# Quickstart: Function Calling and Tools

This notebook demonstrates how to use function calling (tools) with v-router across different LLM providers.

## What is Function Calling?

Function calling allows LLMs to interact with external tools and APIs by generating structured function calls. v-router provides:
- **Unified tool interface** across all providers (Anthropic, OpenAI, Google, Azure)
- **Automatic tool inheritance** for fallback models
- **Type-safe tool definitions** using Pydantic schemas
- **Consistent response format** for tool calls across providers

## Core Tool Components

### Request Models
- **`ToolCall`**: Definition of a single tool with name, description, and input schema
- **`Tools`**: Collection of multiple tools to pass to the LLM
- **`LLM`**: Configuration that includes tools alongside model parameters

### Response Models
- **`Response.tool_calls`**: List of tool calls made by the model
- **`ToolUse`**: Individual tool call with name, ID, and parsed args
- **`Response.content`**: Text content alongside tool calls
- **Unified format**: Same structure across all providers

## Basic Function Calling

Let's start with a simple weather tool example to demonstrate the fundamentals.

In [2]:
from pydantic import BaseModel, Field
from v_router import Client, LLM
from v_router.classes.tools import ToolCall
from v_router.classes.messages import HumanMessage, ToolMessage, AIMessage

# Define the tool schema using Pydantic
class WeatherQuery(BaseModel):
    """Schema for weather query parameters."""
    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")
    units: str = Field("fahrenheit", description="Temperature units: fahrenheit or celsius")

# Create a tool definition
weather_tool = ToolCall(
    name="get_weather",
    description="Get the current weather in a given location",
    input_schema=WeatherQuery.model_json_schema()
)

# Pass tools as a list
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=[weather_tool]  # Clean and simple
)

# Alternative syntax (also works):
# tools=Tools(tools=[weather_tool])

client = Client(llm_config)

# Make a request that should trigger the tool
response = await client.messages.create(
    messages=[
        HumanMessage(content="What's the weather like in San Francisco?")
    ]
)

print(f"Response model: {response.model}")
print(f"Provider: {response.provider}")

# Check the unified response format
print(f"\nContent blocks ({len(response.content)}):")
for i, content in enumerate(response.content):
    print(f"  Block {i}: type={content.type}, role={content.role}")
    if content.type == "text":
        print(f"    Text: {content.text}")

print(f"\nTool uses ({len(response.tool_calls)}):")
for i, tool_calls in enumerate(response.tool_calls):
    print(f"  Tool {i}: {tool_calls.name}")
    print(f"    ID: {tool_calls.id}")
    print(f"    args: {tool_calls.args}")

print(f"\nUsage:")
print(f"  Input tokens: {response.usage.input_tokens}")
print(f"  Output tokens: {response.usage.output_tokens}")

2025-06-13 13:54:20,669 - v_router.router - INFO - Trying primary model: claude-sonnet-4-20250514 on anthropic


Response model: claude-sonnet-4-20250514
Provider: anthropic

Content blocks (1):
  Block 0: type=text, role=assistant
    Text: I'll check the current weather in San Francisco for you.

Tool uses (1):
  Tool 0: get_weather
    ID: toolu_01A6kz2N79iuZBHpvp6W7UpL
    args: {'location': 'San Francisco, CA'}

Usage:
  Input tokens: 462
  Output tokens: 69


In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two numbers.

    Args:
        a: The first number.
        b: The second number.

    Returns:
        The product of the two numbers.
    """
    return a * b

# Pass tools as a list
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=[multiply]  # Clean and simple
)

# Alternative syntax (also works):
# tools=Tools(tools=[weather_tool])

client = Client(llm_config)

# Make a request that should trigger the tool
response = await client.messages.create(
    messages=[
        HumanMessage(content="What's 10 * 20?")
    ]
)

answers = []

for tool_call in response.tool_calls:
    answers.append(invoke_tool(tool_call))


## Multiple Tools Example

You can define multiple tools for more complex workflows. Each tool is defined with its own Pydantic schema for type safety.

In [3]:
# Define multiple tool schemas
class CalculatorQuery(BaseModel):
    """Schema for calculator operations."""
    operation: str = Field(..., description="The mathematical operation: add, subtract, multiply, divide")
    a: float = Field(..., description="First number")
    b: float = Field(..., description="Second number")

class TimeQuery(BaseModel):
    """Schema for time queries."""
    timezone: str = Field("UTC", description="Timezone (e.g., UTC, America/New_York, Europe/London)")

# Create multiple tools
calculator_tool = ToolCall(
    name="calculator",
    description="Perform basic mathematical operations",
    input_schema=CalculatorQuery.model_json_schema()
)

time_tool = ToolCall(
    name="get_current_time",
    description="Get the current time in a specified timezone",
    input_schema=TimeQuery.model_json_schema()
)

weather_tool = ToolCall(
    name="get_weather",
    description="Get the current weather in a given location",
    input_schema=WeatherQuery.model_json_schema()
)

# Pass multiple tools as a list
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=[calculator_tool, time_tool, weather_tool]
)

# Alternative syntax: tools=Tools(tools=[calculator_tool, time_tool, weather_tool])

client = Client(llm_config)

# Test with a query that could use multiple tools
response = await client.messages.create(
    messages=[
        HumanMessage(content="What's 15 * 23? Also, what time is it in New York?")
    ]
)

print(f"Provider: {response.provider}")
print(f"Model: {response.model}")

# Show the unified response format
print(f"\n📝 Content blocks: {len(response.content)}")
for content in response.content:
    print(f"  - {content.type}: {content.text[:50]}..." if len(content.text) > 50 else f"  - {content.type}: {content.text}")

print(f"\n🔧 Tool uses: {len(response.tool_calls)}")
for tool in response.tool_calls:
    print(f"  - {tool.name}: {tool.args}")

2025-06-13 13:58:37,551 - v_router.router - INFO - Trying primary model: claude-sonnet-4-20250514 on anthropic


Provider: anthropic
Model: claude-sonnet-4-20250514

📝 Content blocks: 1
  - text: I'll help you with both of those questions!

🔧 Tool uses: 2
  - calculator: {'operation': 'multiply', 'a': 15, 'b': 23}
  - get_current_time: {'timezone': 'America/New_York'}


## Cross-Provider Function Calling

The same tools work across different providers! This demonstrates v-router's unified interface.

In [4]:
# Same tools, different provider - OpenAI
llm_config_openai = LLM(
    model_name="gpt-4",
    provider="openai",
    tools=[calculator_tool, time_tool, weather_tool]  # Same syntax across providers
)

client_openai = Client(llm_config_openai)

response_openai = await client_openai.messages.create(
    messages=[
        HumanMessage(content="Calculate 42 divided by 7")
    ]
)

print("=== OpenAI Provider ===")
print(f"Provider: {response_openai.provider}")
print(f"Model: {response_openai.model}")

# Unified response format across all providers!
print(f"\n📝 Content: {len(response_openai.content)} blocks")
for content in response_openai.content:
    if content.type == "text":
        print(f"  Text: {content.text}")

print(f"\n🔧 Tool uses: {len(response_openai.tool_calls)}")
for tool_calls in response_openai.tool_calls:
    print(f"  Tool: {tool_calls.name}")
    print(f"  args: {tool_calls.args}")

# Let's also try with Google
llm_config_google = LLM(
    model_name="gemini-1.5-pro",
    provider="google",
    tools=[calculator_tool]  # Just the calculator for Google
)

client_google = Client(llm_config_google)

response_google = await client_google.messages.create(
    messages=[
        {"role": "user", "content": "What is 8 times 9?"}
    ]
)

print("\n=== Google Provider ===")
print(f"Provider: {response_google.provider}")
print(f"Model: {response_google.model}")

# Same unified format for Google!
print(f"\n📝 Content: {len(response_google.content)} blocks")
for content in response_google.content:
    if content.type == "text":
        print(f"  Text: {content.text}")

print(f"\n🔧 Tool uses: {len(response_google.tool_calls)}")
for tool_calls in response_google.tool_calls:
    print(f"  Tool: {tool_calls.name}")
    print(f"  args: {tool_calls.args}")

print("\n✅ Notice how all providers now return the same Response format!")
print("✅ And the same tools syntax works everywhere!")

2025-06-13 13:58:44,927 - v_router.router - INFO - Trying primary model: gpt-4 on openai
2025-06-13 13:58:47,409 - v_router.router - INFO - Trying primary model: gemini-1.5-pro on google


=== OpenAI Provider ===
Provider: openai
Model: gpt-4-0613

📝 Content: 0 blocks

🔧 Tool uses: 1
  Tool: calculator
  args: {'operation': 'divide', 'a': 42, 'b': 7}

=== Google Provider ===
Provider: google
Model: gemini-1.5-pro

📝 Content: 0 blocks

🔧 Tool uses: 1
  Tool: calculator
  args: {'operation': 'multiply', 'b': 9, 'a': 8}

✅ Notice how all providers now return the same Response format!
✅ And the same tools syntax works everywhere!


## Function Calling with Fallbacks

Tools work seamlessly with the fallback system. If the primary model fails, the backup models will automatically inherit the same tools.

### How Tool Inheritance Works:
1. **Primary Model**: Defines tools in the main LLM configuration
2. **Automatic Inheritance**: Backup models inherit tools from primary (no need to redefine)
3. **Provider Translation**: Tools are automatically translated to each provider's format
4. **Seamless Fallback**: If primary fails, backup models have the same capabilities

In [6]:
from v_router import BackupModel

# Create a configuration with a non-existent primary model and fallbacks
llm_config_with_fallback = LLM(
    model_name="claude-nonexistent",  # This will fail
    provider="anthropic",
    tools=[calculator_tool, time_tool, weather_tool],  # ✨ Simplified API in primary config
    backup_models=[
        BackupModel(
            model=LLM(
                model_name="gpt-4",
                provider="openai"
                # Note: No tools specified here - they'll be inherited from primary
            ),
            priority=1
        ),
        BackupModel(
            model=LLM(
                model_name="gemini-1.5-pro", 
                provider="google"
                # Note: No tools specified here - they'll be inherited from primary
            ),
            priority=2
        )
    ]
)

client_fallback = Client(llm_config_with_fallback)

# This will fail on the primary model but succeed on the fallback with tools intact
response_fallback = await client_fallback.messages.create(
    messages=[
        HumanMessage(content="Please calculate 100 divided by 5")
    ]
)

print(f"Provider used: {response_fallback.provider}")
print(f"Model used: {response_fallback.model}")

# Check if tools were used (unified format)
print(f"\n✅ Tools work seamlessly with fallbacks!")
print(f"Content blocks: {len(response_fallback.content)}")
for content in response_fallback.content:
    if content.type == "text":
        print(f"  Text: {content.text}")

if response_fallback.tool_calls:
    print(f"\nTool calls: {len(response_fallback.tool_calls)}")
    for tool_calls in response_fallback.tool_calls:
        print(f"  Tool: {tool_calls.name}")
        print(f"  args: {tool_calls.args}")
        
print(f"\n💡 The backup model inherited tools from the primary configuration!")
print(f"✨ Using the simplified API: tools=[tool1, tool2, tool3]")

2025-06-13 13:59:16,215 - v_router.router - INFO - Trying primary model: claude-nonexistent on anthropic
2025-06-13 13:59:16,428 - v_router.router - WARNING - Primary model failed: Error code: 404 - {'type': 'error', 'error': {'type': 'not_found_error', 'message': 'model: claude-nonexistent'}}
2025-06-13 13:59:16,430 - v_router.router - INFO - Trying backup model: gpt-4 on openai


Provider used: openai
Model used: gpt-4-0613

✅ Tools work seamlessly with fallbacks!
Content blocks: 0

Tool calls: 1
  Tool: calculator
  args: {'operation': 'divide', 'a': 100, 'b': 5}

💡 The backup model inherited tools from the primary configuration!
✨ Using the simplified API: tools=[tool1, tool2, tool3]


## Handling Tool Responses

In a real application, you'd want to execute the tools and provide results back to the model. Here's a complete example of the full tool calling workflow.

## Complete Tool Calling Workflow with ToolMessage

The new `ToolMessage` class enables seamless tool result handling across all providers. Here's how it works with different providers:

In [7]:
# Demonstrate cross-provider tool message support
providers_to_test = [
    ("anthropic", "claude-sonnet-4-20250514"),
    ("openai", "gpt-4"),
    ("google", "gemini-1.5-pro")
]

for provider_name, model_name in providers_to_test:
    print(f"\n{'='*60}")
    print(f"Testing {provider_name.upper()} Provider")
    print(f"{'='*60}")
    
    # Create client with the provider
    llm_config = LLM(
        model_name=model_name,
        provider=provider_name,
        tools=[calculator_tool]
    )
    client = Client(llm_config)
    
    # Initial request
    messages = [
        HumanMessage("What's 144 divided by 12?")
    ]
    
    response = await client.messages.create(messages=messages)
    
    print(f"Initial response from {provider_name}:")
    if response.content:
        print(f"  Text: {response.content[0].text}")
    
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        print(f"  Tool call: {tool_call.name} with ID: {tool_call.id}")
        
        # Add assistant response
        messages.append({
            AIMessage(
                content=response.content[0].text if response.content else ""
            )
        })
        
        # Execute tool
        result = execute_tool(tool_call.name, tool_call.args)
        
        # Add tool result with ToolMessage
        tool_msg = ToolMessage(
            content=f"Calculation complete: {result}",
            tool_call_id=tool_call.id
        )
        messages.append(tool_msg)
        
        # Get final response
        final = await client.messages.create(messages=messages)
        
        print(f"\nFinal response after tool execution:")
        if final.content:
            print(f"  {final.content[0].text}")
        
        print(f"\n✅ ToolMessage worked perfectly with {provider_name}!")
    
print("\n🎉 ToolMessage provides a unified interface across all providers!")

2025-06-13 13:59:58,695 - v_router.router - INFO - Trying primary model: claude-sonnet-4-20250514 on anthropic



Testing ANTHROPIC Provider
Initial response from anthropic:
  Text: I'll calculate 144 divided by 12 for you.
  Tool call: calculator with ID: toolu_01Kb9wuVwCMD6rmrCvFKAbM5


NameError: name 'execute_tool' is not defined

In [ ]:
import json
from datetime import datetime
from v_router.classes.messages import ToolMessage, HumanMessage

def execute_tool(tool_name: str, tool_input: dict) -> str:
    """Simulate executing tools and returning results."""
    if tool_name == "calculator":
        operation = tool_input["operation"]
        a = tool_input["a"]
        b = tool_input["b"]
        
        if operation == "add":
            result = a + b
        elif operation == "subtract":
            result = a - b
        elif operation == "multiply":
            result = a * b
        elif operation == "divide":
            result = a / b if b != 0 else "Error: Division by zero"
        else:
            result = "Error: Unknown operation"
            
        return f"The result of {a} {operation} {b} is {result}"
    
    elif tool_name == "get_current_time":
        # Simulate getting current time
        return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} (simulated)"
    
    elif tool_name == "get_weather":
        location = tool_input["location"]
        units = tool_input.get("units", "fahrenheit")
        # Simulate weather data
        return f"The weather in {location} is 72°F (22°C), sunny with light winds (simulated)"
    
    return "Tool execution not implemented"

# Create a simple calculator tool for this example
simple_calc_tool = ToolCall(
    name="calculator", 
    description="Perform basic math operations",
    input_schema=CalculatorQuery.model_json_schema()
)

# ✨ Simplified API in action!
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=[simple_calc_tool]  # Much cleaner than Tools(tools=[simple_calc_tool])
)

client = Client(llm_config)

# Initial request
messages = [
    HumanMessage("What is 25 times 4?")
]

response = await client.messages.create(messages=messages)

print("Initial response:")
print(f"Provider: {response.provider}")
print(f"Model: {response.model}")

# Display content blocks
print(f"\nContent blocks: {len(response.content)}")
for content in response.content:
    if content.type == "text":
        print(f"  Text: {content.text}")

# Check if there are tool calls in the unified format
if response.tool_calls:
    print(f"\nFound {len(response.tool_calls)} tool call(s):")
    
    # Build the complete conversation including assistant response and tool calls
    # First, add the assistant's complete response (text + tool calls)
    assistant_message = {
        "role": "assistant",
        "content": response.content[0].text if response.content else ""
    }
    
    # Add tool calls to the assistant message if using dict format
    # For Message objects, the tool calls are handled separately
    messages.append(assistant_message)
    
    # Execute each tool and add results using ToolMessage
    for tool_call in response.tool_calls:
        print(f"\nExecuting tool: {tool_call.name}")
        print(f"Tool ID: {tool_call.id}")
        print(f"Input: {tool_call.args}")
        
        # Execute the tool
        result = execute_tool(tool_call.name, tool_call.args)
        print(f"Result: {result}")
        
        # Add tool result using ToolMessage - v-router will handle provider-specific formatting
        tool_message = ToolMessage(
            content=result,
            tool_call_id=tool_call.id,
            status="success"
        )
        messages.append(tool_message)
    
    # Get final response with tool results
    print("\n--- Sending tool results back to the model ---")
    final_response = await client.messages.create(messages=messages)
    
    print(f"\nFinal response with tool results:")
    print(f"Provider: {final_response.provider}")
    print(f"Content blocks: {len(final_response.content)}")
    for content in final_response.content:
        if content.type == "text":
            print(f"  Text: {content.text}")
else:
    print("\nNo tool calls found in response")

print("\n✅ Tool messages work seamlessly across providers!")
print("✨ v-router automatically converts ToolMessage to the correct format for each provider:")
print("   - OpenAI: role='tool' with tool_call_id")
print("   - Anthropic: role='user' with tool_result content block")
print("   - Google: role='user' with function_response part")

## Advanced Tool Patterns

Let's explore some advanced patterns for tool usage:

In [ ]:
# Complex tool with nested data structures
from typing import List, Optional

class SearchResult(BaseModel):
    """Individual search result."""
    title: str
    url: str
    snippet: str

class WebSearchQuery(BaseModel):
    """Schema for web search operations."""
    query: str = Field(..., description="Search query string")
    max_results: int = Field(5, description="Maximum number of results to return")
    include_snippets: bool = Field(True, description="Whether to include content snippets")
    domains: Optional[List[str]] = Field(None, description="Specific domains to search within")

class DataAnalysisQuery(BaseModel):
    """Schema for data analysis operations."""
    data_source: str = Field(..., description="Path or identifier for the data source")
    analysis_type: str = Field(..., description="Type of analysis: summary, correlation, trend, distribution")
    columns: Optional[List[str]] = Field(None, description="Specific columns to analyze")
    filters: Optional[dict] = Field(None, description="Filters to apply to the data")

# Create advanced tools
web_search_tool = ToolCall(
    name="web_search",
    description="Search the web for information and return relevant results",
    input_schema=WebSearchQuery.model_json_schema()
)

data_analysis_tool = ToolCall(
    name="analyze_data",
    description="Perform statistical analysis on datasets",
    input_schema=DataAnalysisQuery.model_json_schema()
)

# ✨ Simplified API with complex tools!
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=[calculator_tool, weather_tool, web_search_tool, data_analysis_tool]  # Clean list syntax!
)

# Compare with old verbose way:
# tools=Tools(tools=[calculator_tool, weather_tool, web_search_tool, data_analysis_tool])

client = Client(llm_config)

response = await client.messages.create(
    messages=[
        {
            "role": "user", 
            "content": "I need to research machine learning trends. Can you search for recent ML developments and also calculate what 15% of 2000 is?"
        }
    ]
)

print("=== Advanced Tool Usage ===")
print(f"Provider: {response.provider}")
print(f"Model: {response.model}")

print(f"\n📝 Content: {len(response.content)} blocks")
for content in response.content:
    if content.type == "text":
        print(f"  Text: {content.text[:100]}..." if len(content.text) > 100 else f"  Text: {content.text}")

print(f"\n🔧 Tool calls: {len(response.tool_calls)}")
for tool_calls in response.tool_calls:
    print(f"  Tool: {tool_calls.name}")
    print(f"  args: {json.dumps(tool_calls.args, indent=2)}")

print(f"\n💡 Complex tools with nested schemas work seamlessly!")
print(f"✨ Using the clean syntax: tools=[tool1, tool2, tool3, tool4]")

## Understanding the Tool Response Format

Let's examine the unified tool response format in detail:

## Controlling Tool Usage with tool_choice

v-router now supports fine-grained control over when and how tools are used through the `tool_choice` parameter. This allows you to:

- **Force specific tools** to be called
- **Require any tool** to be used
- **Prevent tool usage** entirely
- **Let the model decide** (default behavior)

### Tool Choice Options:

1. **`None` or `"auto"`** (default): Model decides whether to use tools
2. **`"any"`**: Model must use one of the provided tools  
3. **`"none"`**: Model is prevented from using tools
4. **`str` (tool name)**: Force the model to use a specific tool
5. **`dict`**: Provider-specific format for advanced control

### Example 1: Force a Specific Tool

When you want to guarantee that a specific tool is used, regardless of the user's input:

In [ ]:
# Force the model to use the weather tool
llm_config_force_weather = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=[weather_tool, calculator_tool, time_tool],  # ✨ Simplified list syntax
    tool_choice="get_weather"  # Force this specific tool
)

client_force = Client(llm_config_force_weather)

# Even though we ask about math, the model will be forced to use the weather tool
response = await client_force.messages.create(
    messages=[
        {"role": "user", "content": "Hello! Can you help me with some math?"}
    ]
)

print("=== Forced Tool Usage ===")
print(f"Provider: {response.provider}")
print(f"\n📝 Content:")
for content in response.content:
    if content.type == "text":
        print(f"  {content.text}")

print(f"\n🔧 Tool calls (forced):")
for tool_calls in response.tool_calls:
    print(f"  Tool: {tool_calls.name} (forced to use weather tool)")
    print(f"  args: {tool_calls.args}")

print("\n✅ The model was forced to use the weather tool even though the user asked about math!")
print("✨ Using clean syntax: tools=[weather_tool, calculator_tool, time_tool]")

### Example 2: Require Any Tool Usage

When you want to ensure the model uses one of the available tools, but let it choose which one:

In [ ]:
# Require the model to use any of the available tools
llm_config_require_any = LLM(
    model_name="gpt-4",
    provider="openai",
    tools=[weather_tool, calculator_tool, time_tool],  # ✨ Clean list syntax
    tool_choice="any"  # Must use one of the provided tools
)

client_require = Client(llm_config_require_any)

# Ask a general question - the model will choose an appropriate tool
response = await client_require.messages.create(
    messages=[
        {"role": "user", "content": "Hi there! How are you doing today?"}
    ]
)

print("=== Required Tool Usage ===")
print(f"Provider: {response.provider}")
print(f"\n📝 Content:")
for content in response.content:
    if content.type == "text":
        print(f"  {content.text}")

print(f"\n🔧 Tool calls (required):")
for tool_calls in response.tool_calls:
    print(f"  Tool chosen: {tool_calls.name}")
    print(f"  args: {tool_calls.args}")

print("\n✅ The model was required to use a tool and chose one that seemed most appropriate!")
print("✨ Simplified: tools=[weather_tool, calculator_tool, time_tool]")

### Example 3: Disable Tool Usage

When you want to prevent the model from using tools, even if they're available:

In [ ]:
# Prevent the model from using any tools
llm_config_no_tools = LLM(
    model_name="gemini-1.5-pro",
    provider="google", 
    tools=[weather_tool, calculator_tool, time_tool],  # ✨ Tools available in clean syntax...
    tool_choice="none"  # ...but model is prevented from using them
)

client_no_tools = Client(llm_config_no_tools)

# Ask something that would normally trigger a tool call
response = await client_no_tools.messages.create(
    messages=[
        {"role": "user", "content": "What's 50 times 20? Please calculate this for me."}
    ]
)

print("=== Disabled Tool Usage ===")
print(f"Provider: {response.provider}")
print(f"\n📝 Content:")
for content in response.content:
    if content.type == "text":
        print(f"  {content.text}")

print(f"\n🔧 Tool calls: {len(response.tool_calls)}")
if response.tool_calls:
    for tool_calls in response.tool_calls:
        print(f"  Tool: {tool_calls.name}")
else:
    print("  No tools used (disabled)")

print("\n✅ Tools were disabled, so the model provided a text response instead!")
print("✨ Clean setup: tools=[weather_tool, calculator_tool, time_tool]")

### Example 4: Auto Mode (Default Behavior)

This is the default behavior - the model decides whether to use tools based on the context:

In [ ]:
# Default behavior - model decides whether to use tools
llm_config_auto = LLM(
    model_name="claude-sonnet-4-20250514", 
    provider="anthropic",
    tools=[weather_tool, calculator_tool, time_tool],  # ✨ Simplified syntax
    tool_choice="auto"  # Explicit auto mode (same as None or not specified)
)

client_auto = Client(llm_config_auto)

# Test with different types of queries
test_queries = [
    "Hello! How are you today?",  # Likely no tools
    "What's 15 * 32?",           # Likely calculator tool
    "What time is it in Tokyo?"  # Likely time tool
]

for i, query in enumerate(test_queries):
    print(f"\n=== Auto Mode Test {i+1}: '{query}' ===")
    
    response = await client_auto.messages.create(
        messages=[{"role": "user", "content": query}]
    )
    
    print(f"📝 Response: {response.content[0].text[:60]}..." if response.content else "No text content")
    print(f"🔧 Tools used: {len(response.tool_calls)}")
    
    if response.tool_calls:
        for tool_calls in response.tool_calls:
            print(f"  - {tool_calls.name}: {tool_calls.args}")
    else:
        print("  - No tools (model decided they weren't needed)")

print("\n✅ In auto mode, the model intelligently chooses when to use tools!")
print("✨ Setup: tools=[weather_tool, calculator_tool, time_tool]")

### Example 5: Provider-Specific Format

For advanced use cases, you can use provider-specific tool_choice formats:

In [ ]:
# Provider-specific format examples
print("=== Provider-Specific tool_choice Examples ===\n")

# Anthropic-specific format
llm_anthropic_specific = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic", 
    tools=[calculator_tool],  # ✨ Clean list syntax
    tool_choice={"type": "tool", "name": "calculator"}  # Anthropic-specific format
)

client_anthropic = Client(llm_anthropic_specific)

response = await client_anthropic.messages.create(
    messages=[{"role": "user", "content": "I need help with something"}]
)

print("🔵 Anthropic with provider-specific format:")
print(f"  Tools used: {len(response.tool_calls)}")
if response.tool_calls:
    print(f"  Tool: {response.tool_calls[0].name}")

# OpenAI-specific format  
llm_openai_specific = LLM(
    model_name="gpt-4",
    provider="openai",
    tools=[calculator_tool],  # ✨ Clean list syntax
    tool_choice={"type": "function", "function": {"name": "calculator"}}  # OpenAI-specific format
)

client_openai = Client(llm_openai_specific)

response = await client_openai.messages.create(
    messages=[{"role": "user", "content": "Help me please"}]
)

print("\n🟠 OpenAI with provider-specific format:")
print(f"  Tools used: {len(response.tool_calls)}")
if response.tool_calls:
    print(f"  Tool: {response.tool_calls[0].name}")

print("\n✅ Provider-specific formats give you maximum control!")
print("✨ Both using simplified: tools=[calculator_tool]")

### Tool Choice with Fallbacks

The `tool_choice` parameter works seamlessly with fallback models - backup models inherit the tool choice behavior:

In [ ]:
# Tool choice with fallbacks
llm_with_fallback = LLM(
    model_name="claude-nonexistent-model",  # This will fail
    provider="anthropic",
    tools=[calculator_tool, weather_tool],  # ✨ Simplified list syntax
    tool_choice="calculator",  # Force calculator tool
    backup_models=[
        BackupModel(
            model=LLM(
                model_name="gpt-4",
                provider="openai"
                # Note: tools and tool_choice inherited from primary
            ),
            priority=1
        )
    ]
)

client_fallback = Client(llm_with_fallback)

response = await client_fallback.messages.create(
    messages=[
        {"role": "user", "content": "Hello there!"}
    ]
)

print("=== Tool Choice with Fallbacks ===")
print(f"Provider used: {response.provider} (fallback)")
print(f"Model used: {response.model}")

print(f"\n📝 Content:")
for content in response.content:
    if content.type == "text":
        print(f"  {content.text}")

print(f"\n🔧 Tool calls (inherited forced behavior):")
if response.tool_calls:
    for tool_calls in response.tool_calls:
        print(f"  Tool: {tool_calls.name} (forced via inheritance)")
        print(f"  args: {tool_calls.args}")
else:
    print("  No tools used")

print("\n✅ Backup models inherit tool_choice behavior from the primary configuration!")
print("✨ Primary config: tools=[calculator_tool, weather_tool]")

In [ ]:
# Simple tool call to examine response structure
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=[calculator_tool]  # ✨ Clean and simple!
)

client = Client(llm_config)

response = await client.messages.create(
    messages=[
        {"role": "user", "content": "Calculate 7 * 8"}
    ]
)

print("🔍 Detailed Response Structure Analysis:")
print(f"\n1. Basic Response Info:")
print(f"   ├── Provider: {response.provider}")
print(f"   ├── Model: {response.model}")
print(f"   └── Response Type: {type(response).__name__}")

print(f"\n2. Content Blocks ({len(response.content)} total):")
for i, content in enumerate(response.content):
    print(f"   Block {i}:")
    print(f"   ├── Type: {content.type}")
    print(f"   ├── Role: {content.role}")
    print(f"   └── Text: {content.text[:50]}..." if len(content.text) > 50 else f"   └── Text: {content.text}")

print(f"\n3. Tool Use Blocks ({len(response.tool_calls)} total):")
for i, tool_calls in enumerate(response.tool_calls):
    print(f"   Tool {i}:")
    print(f"   ├── Name: {tool_calls.name}")
    print(f"   ├── ID: {tool_calls.id}")
    print(f"   ├── args Type: {type(tool_calls.args).__name__}")
    print(f"   └── args: {tool_calls.args}")

print(f"\n4. Usage Information:")
print(f"   ├── Input Tokens: {response.usage.input_tokens}")
print(f"   ├── Output Tokens: {response.usage.output_tokens}")
print(f"   └── Total Tokens: {response.usage.input_tokens + response.usage.output_tokens}")

print(f"\n5. Raw Response:")
print(f"   ├── Type: {type(response.raw_response).__name__}")
print(f"   └── Available for provider-specific processing")

print("\n✅ This exact structure works across ALL providers!")
print("✨ Setup with: tools=[calculator_tool]")

## Summary

### Key Features Demonstrated:

✅ **Unified Tool Interface**: Same tool definitions work across Anthropic, OpenAI, Google, and Azure  
✅ **Type-Safe Schemas**: Use Pydantic models for robust tool parameter validation  
✅ **Tool Choice Control**: Force specific tools, require any tool, or disable tools entirely  
✅ **Automatic Tool Inheritance**: Backup models inherit tools and tool_choice from primary configuration  
✅ **Consistent Response Format**: Same tool response structure across all providers  
✅ **Complex Tool Support**: Handle nested data structures and multiple parameters  
✅ **Full Conversation Flow**: Execute tools and continue conversations with results  
✅ **✨ NEW: Simplified API**: Use `tools=[tool1, tool2]` instead of `tools=Tools(tools=[tool1, tool2])`

### New Simplified Tools API:

#### ✨ Before vs After:
```python
# OLD (still works for backward compatibility):
tools=Tools(tools=[weather_tool, calculator_tool])

# NEW (much cleaner!):
tools=[weather_tool, calculator_tool]
```

Both syntaxes work! The new one is just cleaner and more intuitive.

### Tool Request Models:
- **`ToolCall`**: Individual tool definition (name, description, schema)
- **`Tools`**: Collection of tools (can be created automatically from list)
- **`LLM.tools`**: Now accepts both `Tools` object or `List[ToolCall]` 
- **`LLM.tool_choice`**: Control when and how tools are used
- **Pydantic Models**: Type-safe parameter schemas

### Tool Response Models:
- **`Response.tool_calls`**: List of tool calls made by the model
- **`ToolUse`**: Individual tool call with name, ID, and args
- **`Response.content`**: Text content alongside tool calls
- **`Response.usage`**: Token usage including tool call tokens

### Tool Choice Options:
- **`None` or `"auto"`**: Model decides whether to use tools (default)
- **`"any"`**: Model must use one of the provided tools
- **`"none"`**: Model is prevented from using tools
- **`str` (tool name)**: Force model to use a specific tool
- **`dict`**: Provider-specific format for advanced control

### Advanced Patterns:
- **Multi-step Workflows**: Chain tool calls together
- **Complex Schemas**: Use nested data structures and optional parameters
- **Provider Fallback**: Tools work seamlessly with backup models
- **Tool Result Handling**: Full conversation flow with tool execution
- **Forced Tool Usage**: Guarantee specific tool behavior for consistent workflows

### Next Steps:
- Check out `quickstart_models.ipynb` for basic model usage and fallbacks
- Explore custom tool implementations for your specific use cases
- See the full documentation for advanced tool patterns

v-router provides the most comprehensive and unified function calling interface across all major LLM providers!